In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time, TimezoneInfo, TimeDelta
from statsmodels.tsa.stattools import adfuller

from lsst.sitcom import vandv
from lsst.ts.idl.enums import MTM1M3

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient, clipDataToEvent, calcNextDay
from lsst_efd_client import EfdClient
from tqdm import tqdm
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from glob import glob

In [ ]:
file_list=glob("../data/*slew_characterization.csv")

In [ ]:
frame_list=[]
for file in file_list:
    frame_list.append(pd.read_csv(file))

In [ ]:
total_frame=pd.concat(frame_list).iloc[:,1:]

In [ ]:
total_frame.to_csv("../data/summary_frame.csv",index=False)

In [ ]:
total_frame=total_frame[abs(total_frame["el_max_vel"]) < 100]

In [ ]:
sel=(total_frame["inertial_compensation"]==True)
fig,axs=plt.subplots(2,2, dpi=175, figsize=(10,6))
ax=axs[0,0]
ax.scatter(total_frame["el_begin"][~sel], total_frame["el_end"][~sel], s=0.1, label="compensation off")
ax.scatter(total_frame["el_begin"][sel], total_frame["el_end"][sel], s=0.1, label="compensation on")
ax.set_xlabel("beginning elevation")
ax.set_ylabel("ending elevation")
ax.legend()

ax=axs[0,1]
ax.scatter(total_frame["el_begin"][~sel], total_frame["el_max_vel"][~sel], s=0.1, label="compensation off")
ax.scatter(total_frame["el_begin"][sel], total_frame["el_max_vel"][sel], s=0.1, label="compensation on")
ax.set_xlabel("beginning elevation")
ax.set_ylabel("max velocity elevation\n[deg/s]")

ax=axs[1,0]
ax.scatter(Time(total_frame["time_begin"][~sel],format="unix").datetime,total_frame["el_begin"][~sel],  s=0.1, label="compensation off")
ax.scatter(Time(total_frame["time_begin"][sel],format="unix").datetime, total_frame["el_begin"][sel], s=0.1, label="compensation on")
ax.set_xlabel("date")
ax.set_ylabel("beginning elevation")
ax.set_xticklabels(labels=ax.get_xticklabels(),rotation=45)

axs[1,1].axis('off')